## BRAIx - AI integration analysis

In [ ]:
# Install the analysis package
install.packages("BRAIxMOP_0.2.7.2_R_x86_64-pc-linux-gnu.tar.gz", repos = NULL)
packageVersion("BRAIxMOP")

### Setup

In [ ]:
#Setup
# Set up paths
data_path <- "~/data-source/"
project_path <- "./"
setwd(project_path)

# Set up libraries and helper functions
library(BRAIxMOP)
library(dplyr)
source("renv/activate.R")
source("helpers.R")

# Set up working folder
root_path <- "temp"
input_path <- sprintf("./%s/data", root_path)
output_path <- sprintf("./%s/output", root_path)
for (p in c(root_path, input_path, output_path)) {
    if (!dir.exists(p)) dir.create(p)
}
output <- \(x = "") file.path(output_path, x)
message("Input path: ", input_path)
message("Output path: ", output_path)

# Set up source data
for (f in c("reader_test.csv", "reader_dev.csv", "model_test.csv", "model_dev.csv")) {
    if (file.exists(file.path(data_path, f)) && !file.exists(file.path(input_path, f))) {
        file.copy(file.path(data_path, f), file.path(input_path, f))
    }
}

### Prepare data for simulation

In [ ]:
#Set up simulation data (test)
setup(input_path, output_path, is_test = TRUE)

In [ ]:
#Set up simulation data (dev)
setup(input_path, output_path, is_test = FALSE)

In [ ]:
#Compute reader performance (dev)
# Compute reader performance
get_reader_performance_for_matching(
  reader_file = output("reader_df-dev.RDS"),
  output_directory = output_path,
  is_test = FALSE
) |> invisible()

In [ ]:
#Compute reader performance (test)
# Needed for the ROC figure for comparison
get_reader_performance_for_matching(
  reader_file = output("reader_df.RDS"),
  output_directory = output_path,
  is_test = TRUE
) |> invisible()

In [ ]:
#Find AI threshold to match reader performance on dev (for AI replacement)
# Get thresholds
get_ai_threshold_and_performance(
  readRDS(output("model_df-dev.RDS")),
  readRDS(output("reader_performance-dev.RDS")),
  output_path, F, sort(c(seq(0, 0.9, 0.02), seq(0.9, 0.99, 0.001)))
) |> invisible()

In [ ]:
#Get thresholds for AI band-pass
consensus_dev <- readRDS(output("reader_df-dev.RDS")) |>
  filter(reader_number == 1) |>
  get_consensus_performance()

get_ai_threshold_and_performance_bandpass(
  readRDS(output("model_df-dev.RDS")),
  consensus_dev,
  output_path,
  readRDS(output("accession_df-dev.RDS")),
  readRDS(output("reader_df-dev.RDS")),
  seq(0, 0.99, 0.01)
) |> invisible()

In [ ]:
#Inspect reader and AI performance
env <- list()
env$reader_performance_dev <- readRDS(output("reader_performance-dev.RDS"))
env$reader_performance_dev$mean_reader$weight <- NULL
env$reader_performance_dev$mean_reader$tpr <- NULL
env$reader_performance_dev$mean_reader$fpr <- NULL
message("Reader performance")
print(env$reader_performance_dev)

env$ai_performance <- readRDS(output("AI_performance.RDS"))
env$ai_performance$model_thresholded_df <- NULL
env$ai_performance$search_log <- NULL
env$ai_performance$search_grid <- NULL
message("AI standalone performance")
print(env$ai_performance)

env$ai_performance_band_pass <- readRDS(output("AI_performance_bandpass.RDS"))
env$ai_performance_band_pass$search_log <- NULL
message("AI band-pass performance")
print(env$ai_performance_band_pass)

In [ ]:
#Plot summary of data
# Load data from files
model_df_roc <- roc(readRDS(output("model_df.RDS")))
model_df_dev_roc <- roc(readRDS(output("model_df-dev.RDS")))

reader_df_dev <- readRDS(output("reader_df-dev.RDS"))
consensus_dev <- reader_df_dev |>
  filter(reader_number == 1) |>
  get_consensus_performance()

reader_df <- readRDS(output("reader_df.RDS"))
consensus <- reader_df |>
  filter(reader_number == 1) |>
  get_consensus_performance()

reader_perf <- readRDS(output("reader_performance.RDS"))
reader_perf_dev <- readRDS(output("reader_performance-dev.RDS"))
ai_perf <- readRDS(output("AI_performance.RDS"))
ai_perf_bp <- readRDS(output("AI_performance_bandpass.RDS"))


# Plot
options(repr.plot.width = 11, repr.plot.height = 10)
par(las = 1)
lightgray_alpha <- rgb(0, 0, 0, max = 255, alpha = 10, names = "lightgray_alpha")

# Model
plot(model_df_roc, cex.axis = 1.75, cex.lab = 1.75)
lines(model_df_dev_roc, col = "blue")
with(ai_perf$model_performance, 
     points(TNR, TPR, pch = 19, col = "orange", cex = 2))
with(ai_perf_bp$model_performance,
     points(TNR, TPR, pch = 15, col = "brown", cex = 2))

# Human
# (dev set)
with(reader_perf_dev$mean_reader, 
     points(1 - fpr, tpr, cex = weight * 100, col = lightgray_alpha, pch = 19))
with(reader_perf_dev$mean_reader, 
     points(1 - weighted_fpr, weighted_tpr, col = "pink", pch = 19, cex = 2))
with(reader_perf_dev$mean_reader_by_position, 
     points(1 - weighted_fpr, weighted_tpr, col = "lightblue", pch = 17, cex = 1.1))
with(consensus_dev, points(TNR, TPR, pch = 15, col = "green", cex = 2))

# (test set)
with(reader_perf$mean_reader, 
     points(1 - fpr, tpr, cex = weight * 100, col = lightgray_alpha, pch = 19))
with(reader_perf$mean_reader, 
     points(1 - weighted_fpr, weighted_tpr, col = "red", pch = 19, cex = 2))
with(reader_perf$mean_reader_by_position, 
     points(1 - weighted_fpr, weighted_tpr, col = "blue", pch = 17, cex = 1.1))
with(consensus, points(TNR, TPR, pch = 15, col = "darkgreen", cex = 2))
    
legend(
    "bottomright",
    lwd = c(1, 1, NA, NA, NA, NA, NA, NA, NA),
    pch = c(NA, NA, 19, 15, 15, 15, 19, 19, 17),
    c(
        sprintf("Model ROC (test) - AUC: %.2f", pROC::auc(model_df_roc)), 
        sprintf("Model ROC (dev) - AUC: %.2f", pROC::auc(model_df_dev_roc)),
        "AI standalone (matched)", 
        "AI band-pass",
        "Consensus (dev)",
        "Consensus (test)",
        "Readers",
        "Mean reader (weighted)",
        "Mean reader by position (weighted)"
    ),
    col = c("black", "blue", "orange", "brown", "green", "darkgreen", lightgray_alpha, "red", "blue"),
    cex = 1
)

In [ ]:
#Inspect reader performance by position
reader_perf_dev$mean_reader_by_position
reader_perf$mean_reader_by_position

## Simulation studies

In [ ]:
#Check pre-processed data for simulation
required_files <- c(
    "model_df.RDS", "model_df-dev.RDS",
    "reader_df.RDS", "reader_df-dev.RDS",
    "accession_df.RDS", "accession_df-dev.RDS",
    "AI_performance.RDS", "AI_performance_bandpass.RDS"
)

stopifnot(
    all(unlist(Map(required_files, f = \(x) file.exists(output(x)))))
)

In [ ]:
#Load pre-processed data for simulation
accession_df <- readRDS(output("accession_df.RDS"))
model_df <- readRDS(output("model_df.RDS"))
reader_df <- readRDS(output("reader_df.RDS"))

ai_performance <- readRDS(output("AI_performance.RDS"))
ai_performance_bandpass <- readRDS(output("AI_performance_bandpass.RDS"))

In [ ]:
#Model / Data summary
# Organising blocks (empty)

In [ ]:
#Compare reader and AI model
reader_perf <- readRDS(output("reader_performance.RDS"))
reader_perf$mean_reader$weight <- NULL
reader_perf$mean_reader$tpr <- reader_perf$mean_reader$fpr <- NULL
c(reader_perf$mean_reader$weighted_tpr, 1 - reader_perf$mean_reader$weighted_fpr)
ai_performance$model_performance

In [ ]:
#Generate reader ROC
reader_roc(
  reader_perf = reader_df,
  model_pred = model_df
)

### AI band-pass scenario

In [ ]:
#Scenario analysis
# Organising block
target_file <- output("custom.RDS")
if (!file.exists(target_file)) {
    emp_perf <- get_reader_summary(reader_df)$mean_reader_by_position  # Test performance
    emp_tpr_3rd <- emp_perf$weighted_tpr[3]
    emp_fpr_3rd <- emp_perf$weighted_fpr[3]       
    custom <- c(emp_fpr_3rd, emp_tpr_3rd, emp_tpr_3rd, emp_fpr_3rd, emp_fpr_3rd)
    save_RDS(custom, target_file)
} else {
    custom <- readRDS(target_file)
}

set.seed(1234)
seeds <- c(1234, sample(1e8, 1e6))

In [ ]:
#AI band-pass scenario (group)
# Organising block (empty)

In [ ]:
#AI band-pass scenario
# Note that band-pass is deterministic and has no variation
band_pass_result <- run_simulation(
  run_fun = run_AI_bandpass,
  accession_df = accession_df,
  reader_df = reader_df,
  model_df = model_df,
  threshold_1 = ai_performance_bandpass$low_band_threshold,
  threshold_2 = ai_performance_bandpass$high_band_threshold,
  seed = 1234
)
band_pass_result$system_performance

In [ ]:
#Write to file: AI band-pass scenario
save_RDS(band_pass_result,
         output("simulation_band_pass.RDS"),
         "Simulation result of the band-pass scenario")

In [ ]:
#AI band-pass scenario (Extension 1 - Audit)
set.seed(1234)
ai_performance_bandpass <- readRDS(output("AI_performance_bandpass.RDS"))
model_l_branch_tdf <- apply_manufacturer_threshold(model_df, ai_performance_bandpass$low_band_threshold)
model_u_branch_tdf <- apply_manufacturer_threshold(model_df, ai_performance_bandpass$high_band_threshold)

iter <- 10
s <- seq(0, 1, 0.025)
res_band_pass_audit_all_readers = do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter)
    cbind(p = s,
    do.call(rbind, purrr::map(s, function(p) {
        model_q_tdf <- apply_manufacturer_threshold(model_df, ai_performance$threshold)
        res <- AI_band_pass_audit_all_readers(accession_df, reader_df, model_l_branch_tdf, model_u_branch_tdf, model_q_tdf,
                                  scores = model_q_tdf |> select(episode_id, episode_prediction),
                                  probs = c(p, p, 0, 0), require_sim = p > 0, custom = custom)
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
res_band_pass_audit_all_readers

In [ ]:
#AI band-pass scenario (Extension 2 - Learn)
set.seed(1234)
ai_performance_bandpass <- readRDS(output("AI_performance_bandpass.RDS"))
ai_performance_bandpass$low_band_threshold
ai_performance_bandpass$high_band_threshold

model_l_branch_tdf <- apply_manufacturer_threshold(model_df, ai_performance_bandpass$low_band_threshold)
model_u_branch_tdf <- apply_manufacturer_threshold(model_df, ai_performance_bandpass$high_band_threshold)

model_df |>
    group_by(manufacturer) |>
    summarise(thresholds = quantile(episode_prediction, 0.98)) |>
    IRdisplay::display()


iter <- 10
s <- seq(0, 1, 0.025)
res_band_pass_learn_plus_all_readers = do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter)
    cbind(p = s,
    do.call(rbind, purrr::map(s, function(p) {
        model_q_tdf <- apply_manufacturer_threshold(model_df, ai_performance$threshold)
        res <- AI_band_pass_learn_all_readers(accession_df, reader_df, model_l_branch_tdf, model_u_branch_tdf, model_q_tdf,
                                  probs = c(p, p, 0, 0), method = "correction", 
                                  require_sim = p > 0 && p < 1, custom = custom)
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
res_band_pass_learn_plus_all_readers


res_band_pass_learn_minus_all_readers = do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter)
    cbind(p = s,
    do.call(rbind, purrr::map(s, function(p) {
        model_q_tdf <- apply_manufacturer_threshold(model_df, ai_performance$threshold)
        res <- AI_band_pass_learn_all_readers(accession_df, reader_df, model_l_branch_tdf, model_u_branch_tdf, model_q_tdf,
                                  probs = c(p, p, 0, 0), method = "error", 
                                  require_sim = p > 0 && p < 1, custom = custom)
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
res_band_pass_learn_minus_all_readers

In [ ]:
#AI band-pass scenario (Extension) [Plot]
save <- TRUE
options(repr.plot.width = 12, repr.plot.height = 10)
par(las = 1)
plot(roc(model_df), cex.axis = 1.5, main = "Band-Pass human-AI interaction",
     xlim = c(1, 0.7), ylim = c(0.7, 1),
     mar = c(5.1, 6.1, 4.1, 2.1),
     cex.lab = 1.5,
     mgp = c(4, 1, 0))

with(res_band_pass_audit_all_readers, points(TNR, TPR, cex = 1, col = "brown", pch = 19))
with(res_band_pass_learn_plus_all_readers, points(TNR, TPR, cex = 1, col = "green", pch = 19))
with(res_band_pass_learn_minus_all_readers, points(TNR, TPR, cex = 1, col = "red", pch = 19))

# Baseline band-pass
res_bp <- readRDS(output("simulation_band_pass.RDS"))
with(res_bp$system_performance,
     points(TNR, TPR, pch = 15, col = "green", cex = 3))

# Consensus point
res_baseline <- baseline(accession_df, reader_df)
with(scenario_summary(res_baseline)$performance,
     points(TNR, TPR, pch = 15, col = "red", cex = 3))

legend("bottomright", c("Consensus", "Band-pass (no interaction)", "Band-pass (Audit)", "Band-pass (Learn+)", "Band-pass (Learn-)"), 
       col = c("red", "green", "brown", "green", "red"), 
       pch = c(15, 15, 19, 19, 19))


# Extract data
plot_bp_extension <- rbind(
    cbind(name = "audit", res_band_pass_audit_all_readers),
    cbind(name = "learn_plus", res_band_pass_learn_plus_all_readers),
    cbind(name = "learn_minus", res_band_pass_learn_minus_all_readers)
)
plot_bp_extension

plot_band_pass_extension <- plot_bp_extension
if (save) write.csv(plot_band_pass_extension, output("plot_band_pass_extension.csv"), row.names = FALSE)

plot_band_pass_extension_extra <- cbind(
    name = c("band-pass", "baseline"),
    rbind(
      res_bp$system_performance,
      scenario_summary(res_baseline)$performance
    )
)
if (save) write.csv(plot_band_pass_extension_extra, output("plot_band_pass_extension_extra.csv"), row.names = FALSE)

### AI replacement scenario

In [ ]:
#AI replacement scenario (group)
# Organising block

In [ ]:
#AI replacement scenario (with bootstrap)
result <- run_simulation(
  run_AI_replacement,
  accession_df = accession_df,
  reader_df = reader_df,
  model_df = model_df,
  threshold = ai_performance$threshold,
  seed = 1234,
  custom = c(emp_fpr_3rd, emp_tpr_3rd, emp_tpr_3rd, emp_fpr_3rd, emp_fpr_3rd)
)

result_n <- run_simulation_bootstrap(
  run_AI_replacement,
  accession_df = accession_df,
  reader_df = reader_df,
  model_df = model_df,
  threshold = ai_performance$threshold,
  custom = c(emp_fpr_3rd, emp_tpr_3rd, emp_tpr_3rd, emp_fpr_3rd, emp_fpr_3rd),
  run_parallel = FALSE,
  seed = 1234,
  bootstrap_n = 1000
)

In [ ]:
#Write to file: AI replacement scenario (with bootstrap)
saveRDS(result_n, output("simulation_replacement_original.RDS"))

In [ ]:
#AI replacement scenario (Extension: human-AI interaction)
# Extension: Human-AI interaction
props <- seq(-1, 1, 0.2)
result_n_interaction <- AI_replacement_human_ai_interaction_bootstrap(
  props,
  accession_df = accession_df,
  reader_df = reader_df,
  model_df = model_df,
  threshold = ai_performance$threshold,
  custom = c(emp_fpr_3rd, emp_tpr_3rd, emp_tpr_3rd, emp_fpr_3rd, emp_fpr_3rd),
  bootstrap_n = 1000,
  seed = 1234,
  run_parallel = FALSE
)

In [ ]:
#Write to file: AI replacement scenario (Extension: human-AI interaction)
saveRDS(result_n_interaction, output("simulation_replacement_original_interaction.RDS"))

In [ ]:
#AI replacement scenario (Extension: audit + learn)
set.seed(1234)
iter <- 10
s <- seq(0, 1, 0.025)

message("AI independent audit")
res_AI_independent_audit_summary <- do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter) 
    cbind(iter = idx, p = s,
    do.call(rbind, purrr::map(s, function(p) {
        model_tdf <- apply_manufacturer_threshold(model_df, ai_performance$threshold)
        res <- AI_independent_audit(accession_df, reader_df, model_tdf, 
                                    scores = model_tdf |> select(episode_id, episode_prediction),
                                    probs = c(p, p, 0, 0), custom)
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
res_AI_independent_audit_summary

message("AI independent learn+")
res_AI_independent_learn_plus_summary <- do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter) 
    cbind(iter = idx, p = s,
    do.call(rbind, purrr::map(s, function(p) {
        model_tdf <- apply_manufacturer_threshold(model_df, ai_performance$threshold)
        res <- AI_independent_learn(accession_df, reader_df, model_tdf, 
                                    probs = c(p, p, 0, 0), "correction", custom) 
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
res_AI_independent_learn_plus_summary

message("AI independent learn-")
res_AI_independent_learn_minus_summary <- do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter) 
    cbind(iter = idx, p = s,
    do.call(rbind, purrr::map(s, function(p) {
        model_tdf <- apply_manufacturer_threshold(model_df, ai_performance$threshold)
        res <- AI_independent_learn(accession_df, reader_df, model_tdf, 
                                    probs = c(p, p, 0, 0), "error", custom)
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
res_AI_independent_learn_minus_summary

In [ ]:
#AI replacement scenario (Extension) [Plot]
save <- TRUE
options(repr.plot.width = 12, repr.plot.height = 10)
par(las = 1)
plot(roc(model_df), cex.axis = 1.5, 
     main = "Replacement human-AI interaction",
     xlim = c(1, 0.7), ylim = c(0.7, 1.0),
     mar = c(5.1, 6.1, 4.1, 2.1),
     cex.lab = 1.5,
     mgp = c(4, 1, 0))

pt_size <- 1
with(res_AI_independent_learn_plus_summary, {
    points(TNR, TPR, col = "green", pch = 19, cex = pt_size)
})

with(res_AI_independent_audit_summary, {
    points(TNR, TPR, col = "brown", pch = 19, cex = pt_size)
})

with(res_AI_independent_learn_minus_summary, {
    # lines(TNR, TPR)
    points(TNR, TPR, col = "red", pch = 19, cex = pt_size)
})

res_repl <- readRDS(output("simulation_replacement_original.RDS"))
with(data.frame(lapply(res_repl$performance, mean)),
     points(TNR, TPR, col = "purple", pch = 15, cex = 3))

# Consensus point
res_baseline <- baseline(accession_df, reader_df)
with(scenario_summary(res_baseline)$performance,
     points(TNR, TPR, pch = 15, col = "red", cex = 3))

legend(
    "bottomright", 
    c("Consensus", 
      "Replacement (no interaction)", 
      "Replacement (Audit)",
      "Replacement (Learn+)",
      "Replacement (Learn-)"),
    col = c("red", "purple", "brown", "green", "red"),
    pch = c(15, 15, 19, 19, 19),
    cex = 1
)


# Extract data
plot_replacement_extension <- rbind(
    cbind(name = "audit", res_AI_independent_audit_summary),
    cbind(name = "learn_plus", res_AI_independent_learn_plus_summary),
    cbind(name = "learn_minus", res_AI_independent_learn_minus_summary)
)
plot_replacement_extension
if (save) write.csv(plot_replacement_extension, output("plot_replacement_extension.csv"), row.names = FALSE)

plot_replacement_extension_extra <- cbind(
    name = c("replacement", "baseline"),
    rbind(
      data.frame(lapply(res_repl$performance, mean)),
      scenario_summary(res_baseline)$performance
    )
)
if (save) write.csv(plot_replacement_extension_extra, output("plot_replacement_extension_extra.csv"), row.names = FALSE)

### AI triage (MASAI) scenario

In [ ]:
#Triage scenario (group)
model_df_dev <- readRDS(output("model_df-dev.RDS"))

# # Some basic check
quantile(model_df_dev$episode_prediction, 0.9)
quantile(model_df$episode_prediction, 0.9)
sum(model_df$episode_prediction < 0.0959783851896646)  # dev
sum(model_df$episode_prediction < 0.0951128885848447)  # test
table(model_df_dev$manufacturer)
table(model_df$manufacturer)

In [ ]:
#Triage (MASAI) scenario
set.seed(1234)
thr <- get_manufacturer_quantile_threshold(model_df_dev, q = 0.9)
model_tdf <- apply_manufacturer_threshold(model_df, thr)

res_masai <- AI_masai(accession_df, reader_df, model_tdf, 0.5)
data.frame(scenario_summary(res_masai)$performance)

res_masai_multiple <- {
    res_masai <- AI_masai(accession_df, reader_df, model_tdf, 0.5)
    data.frame(scenario_summary(res_masai)$performance)
} |> 
    quote() |> 
    multiple_seeded_runs(seeds[1:100]) |>
    do_rbind()
res_masai_multiple

In [ ]:
#Write to file: Triage (MASAI) scenario
saveRDS(res_masai, output("simulation_masai.RDS"))
saveRDS(res_masai_multiple, output("simulation_masai_multiple.RDS"))

In [ ]:
#Triage scenario (Extension - Learn from AI+)
set.seed(1234)
# All readers modification
iter <- 10
ps <- seq(0, 1, 0.025)
res_masai_improve_summary_all_readers <- do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter) 
    cbind(p = ps,
    do.call(rbind, purrr::map(ps, function(p) {
        thr <- get_manufacturer_quantile_threshold(model_df_dev, 0.9)
        model_tdf_branch <- apply_manufacturer_threshold(model_df, thr)
        model_tdf_flip <- apply_manufacturer_threshold(model_df, ai_performance$threshold)
        res <- AI_masai_learn_from_AI_all_readers(accession_df, reader_df, model_tdf_branch, model_tdf_flip, 
                                                  probs = c(p, p, 0, 0), method = "correction",
                                                  require_sim = (p != 0) && (p != 1), custom = custom)
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
res_masai_improve_summary_all_readers
save_RDS(res_masai_improve_summary_all_readers, output("res_masai_improve_summary_all_readers.RDS"))

In [ ]:
#Triage scenario (Extension - Learn from AI-)
set.seed(1234)
iter <- 10
ps <- seq(0, 1, 0.025)
res_masai_deteriorate_summary_all_readers <- do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter) 
    cbind(p = ps,
    do.call(rbind, purrr::map(ps, function(p) {
        thr <- get_manufacturer_quantile_threshold(model_df_dev, 0.9)
        model_tdf_branch <- apply_manufacturer_threshold(model_df, thr)
        model_tdf_flip <- apply_manufacturer_threshold(model_df, ai_performance$threshold)
        res <- AI_masai_learn_from_AI_all_readers(accession_df, reader_df, model_tdf_branch, model_tdf_flip, 
                                                  probs = c(p, p, 0, 0), method = "error",
                                                  require_sim = (p != 0) && (p != 1), custom = custom)
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
round(res_masai_deteriorate_summary_all_readers, 4)
save_RDS(res_masai_deteriorate_summary_all_readers, output("res_masai_deteriorate_summary_all_readers.RDS"))

In [ ]:
#Triage scenario (Extension - Audit)
set.seed(1234)
ps <- seq(0, 1, 0.025)
iter <- 10
res_masai_audit_summary_all_readers <- do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter) 
    cbind(p = ps,
    do.call(rbind, purrr::map(ps, function(p) {
        thr <- get_manufacturer_quantile_threshold(model_df_dev, 0.9)
        model_tdf_branch <- apply_manufacturer_threshold(model_df, thr)
        model_tdf_flip <- apply_manufacturer_threshold(model_df, ai_performance$threshold)
        res <- AI_masai_audit_AI_all_readers(accession_df, reader_df, model_tdf_branch, model_tdf_flip, 
                                     # scores = model_df |> select(episode_id, episode_prediction),
                                     scores = model_tdf_flip |> select(episode_id, episode_prediction),
                                     probs = c(p, p, 0, 0), require_sim = (p > 0), custom = custom)
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
res_masai_audit_summary_all_readers
save_RDS(res_masai_audit_summary_all_readers, output("res_masai_audit_summary_all_readers.RDS"))

In [ ]:
#Triage scenario (Extension) [Plot]
save <- TRUE
res_masai <- readRDS(output("simulation_masai.RDS"))
res_masai_multiple <- readRDS(output("simulation_masai_multiple.RDS"))

options(repr.plot.width = 12, repr.plot.height = 10)
par(las = 1)
plot(roc(model_df), cex.axis = 1.5, 
     main = "Triage human-AI interaction",
     xlim = c(1, 0.7), ylim = c(0.7, 1.0),
     mar = c(5.1, 6.1, 4.1, 2.1),
     cex.lab = 1.5,
     mgp = c(4, 1, 0))

with(res_masai_improve_summary_all_readers, {
    points(TNR, TPR, col = "green", pch = 19, cex = 1)
})

with(res_masai_deteriorate_summary_all_readers, {
    points(TNR, TPR, col = "red", pch = 19, cex = 1)
})

with(res_masai_audit_summary_all_readers, {
    points(TNR, TPR, col = "brown", pch = 19, cex = 1)
})

legend(
    "bottomright", 
    c("Consensus", 
      "Triage (no interaction)", 
      "Triage (Audit)",
      "Triage (Learn+)",
      "Triage (Learn-)"),
    col = c("black", "orange", "brown", "green", "red"),
    pch = c(15, 15, 19, 19, 19),
    cex = 1
)

# MASAI baseline
with(scenario_summary(res_masai)$performance,
     points(TNR, TPR, col = "orange", pch = 15, cex = 3))

# Consensus point
res_baseline <- baseline(accession_df, reader_df)
with(scenario_summary(res_baseline)$performance,
     points(TNR, TPR, pch = 15, col = "black", cex = 3))



# Extract data
plot_masai_extension <- rbind(
    cbind(name = "audit", res_masai_audit_summary_all_readers),
    cbind(name = "learn_plus", res_masai_improve_summary_all_readers),
    cbind(name = "learn_minus", res_masai_deteriorate_summary_all_readers)
)
plot_masai_extension

plot_masai_extension_extra <- cbind(
    name = c("triage", "baseline"),
    rbind(
      scenario_summary(res_masai)$performance,
      scenario_summary(res_baseline)$performance
    )
)

plot_masai_extension_extra_2 <- cbind(
    name = "triage",
    res_masai_multiple
)

if (save) {
    write.csv(plot_masai_extension, 
              output("plot_masai_extension.csv"), row.names = FALSE)
    write.csv(plot_masai_extension_extra, 
              output("plot_masai_extension_extra.csv"), row.names = FALSE)
    write.csv(plot_masai_extension_extra_2, 
              output("plot_masai_extension_extra_2.csv"), row.names = FALSE)
}

### Single reader and Sequential readers

In [ ]:
#AI Single - Learn and Audit
model_tdf <- model_df |> apply_manufacturer_threshold(ai_performance$threshold)
ps <- seq(0, 1, 0.025)
iter <- 10
save <- TRUE


set.seed(1234)
res_single <- do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter) 
    cbind(iter = idx, p = ps,
    do.call(rbind, purrr::map(ps, function(p) {
        res <- AI_single(accession_df, reader_df)
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
if (save) write.csv(res_single, output("res_single.csv"), row.names = FALSE)


set.seed(1234)
res_single_learn_plus <- do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter) 
    cbind(iter = idx, p = ps,
    do.call(rbind, purrr::map(ps, function(p) {
        res <- AI_single_learn(accession_df, reader_df, model_tdf, probs = c(p, p, 0, 0), "correction")
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
if (save) write.csv(res_single_learn_plus, output("res_single_learn_plus.csv"), row.names = FALSE)


set.seed(1234)
res_single_learn_minus <- do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter) 
    cbind(iter = idx, p = ps,
    do.call(rbind, purrr::map(ps, function(p) {
        res <- AI_single_learn(accession_df, reader_df, model_tdf, probs = c(p, p, 0, 0), "error")
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
if (save) write.csv(res_single_learn_minus, output("res_single_learn_minus.csv"), row.names = FALSE)


set.seed(1234)
res_single_audit <- do.call(rbind, purrr::map(1:iter, function(idx) {
    message("Processing: ", idx, " / ", iter) 
    cbind(iter = idx, p = ps,
    do.call(rbind, purrr::map(ps, function(p) {
        # res <- AI_single_audit(accession_df, reader_df, model_tdf, scores = model_df, probs = c(p, p, 0, 0))
        res <- AI_single_audit(accession_df, reader_df, model_tdf, scores = model_tdf, probs = c(p, p, 0, 0))
        data.frame(scenario_summary(res)$performance)
    }))
    )
}))
if (save) write.csv(res_single_audit, output("res_single_audit.csv"), row.names = FALSE)


In [ ]:
# Load files from previous runs
res_single <- read.csv(output("res_single.csv"))
res_single_learn_plus <- read.csv(output("res_single_learn_plus.csv"))
res_single_learn_minus <- read.csv(output("res_single_learn_minus.csv"))
res_single_audit <- read.csv(output("res_single_audit.csv"))

In [ ]:
#AI Single - Learn and Audit [Plot]
# Create plots for the AI single
save <- TRUE
options(repr.plot.width = 12, repr.plot.height = 10)
par(las = 1)
plot(roc(model_df), cex.axis = 1.5, 
     main = "Single human-AI interaction",
     xlim = c(1, 0.7), ylim = c(0.5, 1.0),
     mar = c(5.1, 6.1, 4.1, 2.1),
     cex.lab = 1.5,
     mgp = c(4, 1, 0))

pt_size <- 1
with(res_single_learn_plus, {
    points(TNR, TPR, col = "green", pch = 19, cex = pt_size)
})

with(res_single_audit, {
    points(TNR, TPR, col = "brown", pch = 19, cex = pt_size)
})

with(res_single_learn_minus, {
    points(TNR, TPR, col = "red", pch = 19, cex = pt_size)
})

with(res_single, {
    points(TNR, TPR, col = "black", cex = pt_size)
})

# Consensus point
res_baseline <- baseline(accession_df, reader_df)
with(scenario_summary(res_baseline)$performance,
     points(TNR, TPR, pch = 15, col = "red", cex = 3))

legend(
    "bottomright", 
    c("Consensus", 
      "Average Reader (weighted)", 
      "Single (Audit)",
      "Single (Learn+)",
      "Single (Learn-)",
      "Single (AI replacement)"
    ),
    col = c("red", "black", "brown", "green", "red", "purple"),
    pch = c(15, 19, 19, 19, 19, 19),
    cex = 1
)


res_weighted <- readRDS(output("reader_performance.RDS"))

pt_size <- 2
with(res_weighted$mean_reader,
     points(1 - weighted_fpr, weighted_tpr, pch = 19, cex = pt_size))

repl_op <- readRDS(output("AI_performance.RDS"))
with(
  model_df |>
    apply_manufacturer_threshold(repl_op$threshold) |>
    confusion_matrix() |>
    data.frame(),
  points(TNR, TPR, pch = 19, col = "purple", cex = pt_size)
)


# Extract data
plot_single_extension <- rbind(
    cbind(name = "standalone", res_single),
    cbind(name = "audit", res_single_audit),
    cbind(name = "learn_plus", res_single_learn_plus),
    cbind(name = "learn_minus", res_single_learn_minus)
)
plot_single_extension

if (save) write.csv(plot_single_extension, output("plot_single_extension.csv"), row.names = FALSE)

## Tables and Figures

### Result summary

In [ ]:
#Table: Prediction by decile scores
model_df <- readRDS(output("model_df.RDS"))
qs <- quantile(model_df$episode_prediction, seq(0.1, 1, 0.1))

binned_model_pred <- model_df |>
  mutate(binned_score = case_when(
    episode_prediction <= qs[1] ~ 1,
    episode_prediction <= qs[2] ~ 2,
    episode_prediction <= qs[3] ~ 3,
    episode_prediction <= qs[4] ~ 4,
    episode_prediction <= qs[5] ~ 5,
    episode_prediction <= qs[6] ~ 6,
    episode_prediction <= qs[7] ~ 7,
    episode_prediction <= qs[8] ~ 8,
    episode_prediction <= qs[9] ~ 9,
    episode_prediction <= qs[10] ~ 10,
  ))


# Quantile
# qs


# Binned scores
# binned_model_pred |>
#   select(episode_id, episode_outcome, episode_prediction, binned_score)


binned_counts <- binned_model_pred |>
  group_by(binned_score, episode_outcome) |>
  summarise(count = n()) |>
  group_by(episode_outcome) |>
  mutate(total = sum(count),
         weight = count / total,
         weighted_score = weight * binned_score) |>
  arrange(episode_outcome)


binned_counts |>
  print(n = 100)


table_1_df <- binned_counts |>
  tidyr::pivot_wider(id_cols = binned_score,
                     values_from = count,
                     names_from = episode_outcome)
table_1_df[is.na(table_1_df)] <- 0
# table_1_df

for (i in 2:6) {
  col_i <- table_1_df[, i][[1]]
  percent <- round(col_i / sum(col_i) * 100 , 1)
  table_1_df[, i] <- paste(col_i, " (", percent, ")", sep = "")
}
# table_1_df
table_1_df <- table_1_df[, c("binned_score", "1", "2", "0", "3", "4")]
colnames(table_1_df) <- c(
  "Binned AI reader scores",
  "Screen-detected cancer",
  "Interval cancer",
  "Normal",
  "Benign",
  "NSA")
table_1_df


table_1_LaTeX <- xtable::xtable(table_1_df)
table_1_LaTeX |> print()

In [ ]:
#Table: Scenario summary table
# Baseline, Reader replacement, bandpass and MASAI
res_baseline <- baseline(accession_df, reader_df)
res_baseline_econ <- BRAIxMOP:::baseline_economics(scenario_summary(res_baseline))

res_repl <- readRDS(output("simulation_replacement_original.RDS"))

res_bp <- readRDS(output("simulation_band_pass.RDS"))

res_masai <- readRDS(output("simulation_masai.RDS"))
res_masai_econ <- BRAIxMOP:::masai_economics(scenario_summary(res_masai))

message("Generating LaTeX table")
table_2(
  cbind(baseline_summary_col(res_baseline_econ),
        repl_summary_col(res_repl, res_baseline_econ),
        bp_summary_col(res_bp$system_econ_table),
        masai_summary_col(res_masai_econ, res_baseline_econ))
) |> cat()

In [ ]:
#Figure: Individual ROC curve
model_df <- readRDS(output("model_df.RDS"))
reader_df <- readRDS(output("reader_df.RDS"))
accession_df <- readRDS(output("accession_df.RDS"))

# Base plot
par(las = 1)
options(repr.plot.width = 11, repr.plot.height = 10)
plot(roc(model_df), type = 'n')

# Readers points
res_reader <- get_reader_performance(reader_df)
point_size <- 100   # use 10 for example data
with(res_reader,
     points(TNR, TPR, cex = point_size * (1 + P + N) / sum(P + N),
            col = 'lightgray'))

# ROC curve
lines(roc(model_df))

# Consensus point
res_baseline <- baseline(accession_df, reader_df)
with(scenario_summary(res_baseline)$performance,
     points(TNR, TPR, pch = 15, col = "red"))

# Weighted reader point
res_weighted <- readRDS(output("reader_performance.RDS"))
with(res_weighted$mean_reader,
     points(1 - weighted_fpr, weighted_tpr, pch = 19))

In [ ]:
#Figure 2: Flowcharts
# Figure 2: Flowcharts =========================================================
cat(flowchart_A(res_baseline))
cat(flowchart_B(res_repl))
cat(flowchart_C(res_bp))
cat(flowchart_D(res_masai))

In [ ]:
#Figure 3: Scenario ROC curve (data)
# Figure 3: Scenario ROC curve =================================================
# System curve - Replacement, band-pass, MASAI
# MASAI
s <- c(seq(0, 0.9, 0.05), seq(0.91, 0.99, 0.005), seq(0.991, 1, 0.001)) 
res_masai_curve <- masai_curve(s)
write.csv(res_masai_curve, file = output("curve_masai.csv"), row.names = FALSE)


# Band-pass scenario
reader_df_dev <- readRDS(output("reader_df-dev.RDS"))
consensus_dev <- reader_df_dev |>
  filter(reader_number == 1) |>
  get_consensus_performance()
res_bp_thr <- get_ai_threshold_and_performance_bandpass(
  readRDS(output("model_df.RDS")),
  consensus_dev,
  NULL,
  readRDS(output("accession_df.RDS")),
  readRDS(output("reader_df.RDS")),
  seq(0, 0.99, 0.02)
)

# Compute efficient boundary
res_bp_curve <- eff(res_bp_thr$search_log)
write.csv(res_bp_curve, file = output("curve_bandpass.csv"), row.names = FALSE)


# Replacement scenario
specs <- c(seq(0, 0.99, 0.01), seq(0.991, 0.995, 0.001))  # use `specs <- seq(0, 0.99, 0.04)` for example data
res_repl_curve <- data.frame(do.call(rbind, lapply(
  specs,
  function(spec) {
     result <- run_simulation(
       run_AI_replacement,
       accession_df = accession_df,
       reader_df = reader_df,
       model_df = model_df,
       threshold = find_manufacturer_threshold_matching(model_df, "specificities", spec),
       seed = 1234,
       custom = custom
     )
     unlist(result$system_econ)
  }
)))

write.csv(res_repl_curve, file = output("curve_replacement.csv"), row.names = FALSE)

In [ ]:
#Figure 3: Scenario ROC curve (plot)
# Figure 3: Scenario ROC curve =================================================
temp <- list()
res_weighted <- readRDS(output("reader_performance.RDS"))

pt_size <- 2
plot(roc(model_df), lty = 3, xlim = c(1, 0.6), ylim = c(0.6, 1))
with(res_weighted$mean_reader,
     points(1 - weighted_fpr, weighted_tpr, pch = 19, cex = pt_size))

repl_op <- readRDS(output("AI_performance.RDS"))
with(
  temp$plot_repl_op <- model_df |>
    apply_manufacturer_threshold(repl_op$threshold) |>
    confusion_matrix() |>
    data.frame(),
  points(TNR, TPR, pch = 19, col = "green", cex = pt_size)
)

bp_op <- readRDS(output("AI_performance_bandpass.RDS"))
with(
  temp$plot_bp_op_low <- model_df |>
    apply_manufacturer_threshold(bp_op$low_band_threshold) |>
    confusion_matrix() |>
    data.frame(),
  points(TNR, TPR, pch = 19, col = "purple", cex = pt_size)
)
with(
  temp$plot_bp_op_high <- model_df |>
    apply_manufacturer_threshold(bp_op$high_band_threshold) |>
    confusion_matrix() |>
    data.frame(),
  points(TNR, TPR, pch = 19, col = "purple", cex = pt_size)
)

# Add scenario points
with(temp$plot_repl_perf <- data.frame(t(colMeans(res_repl$performance))),
     points(TNR, TPR, pch = 15, col = "green", cex = pt_size))

with(temp$plot_bp <- res_bp$system_performance,
     points(TNR, TPR, pch = 15, col = "purple", cex = pt_size))

with(temp$plot_consensus <- scenario_summary(res_baseline)$performance,
     points(TNR, TPR, pch = 15, col = "red", cex = pt_size))

with(temp$plot_masai <- scenario_summary(res_masai)$performance,
     points(TNR, TPR, pch = 15, col = "orange", cex = pt_size))

legend("bottomright",
       c("Weighted mean reader", "AI replacement operating point", "AI bandpass operating points",
         "Consensus", "AI replacement scenario", "AI band-pass scenario", "MASAI"),
       pch = c(19, 19, 19, 15, 15, 15, 15),
       col = c("black", "green", "purple",
               "red", "green", "purple", "orange"))


result_df <- data.frame(rbind(
  c("Weighted Mean Reader",
    1 - res_weighted$mean_reader$weighted_fpr,
    res_weighted$mean_reader$weighted_tpr),
  c("Replacement OP",
    temp$plot_repl_op$TNR,
    temp$plot_repl_op$TPR),
  c("Band-pass OP low",
    temp$plot_bp_op_low$TNR,
    temp$plot_bp_op_low$TPR),
  c("Band-pass OP high",
    temp$plot_bp_op_high$TNR,
    temp$plot_bp_op_high$TPR),
  c("Censensus",
    temp$plot_consensus$TNR,
    temp$plot_consensus$TPR),
  c("Replacement",
    temp$plot_repl_perf$TNR,
    temp$plot_repl_perf$TPR),
  c("Band-pass",
    temp$plot_bp$TNR,
    temp$plot_bp$TPR),
  c("MASAI",
    temp$plot_masai$TNR,
    temp$plot_masai$TPR)
))
colnames(result_df) <- c("desc", "TNR", "TPR")
result_df
write.csv(result_df, file = output("scenario_roc.csv"), row.names = FALSE)


res_masai_curve <- read.csv(output("curve_masai.csv"))
lines(res_masai_curve$benefit_rate.TN, res_masai_curve$benefit_rate.TP,
      col = "orange")


res_bp_curve <- read.csv(output("curve_bandpass.csv"))
lines(res_bp_curve$TNR, res_bp_curve$TPR, col = "purple")
lines(c(1, max(res_bp_curve$TNR)), c(0, min(res_bp_curve$TPR)), col = "purple")


res_repl_curve <- read.csv(output("curve_replacement.csv"))
local({
  with(res_repl_curve, {
    TNR <- benefit_rate.TN
    TPR <- benefit_rate.TP
    lines(TNR, TPR, col = "green")
    lines(c(1, max(TNR)), c(0, min(TPR)), col = "green")
    lines(c(0, min(TNR)), c(1, max(TPR)), col = "green")
  })
})

In [ ]:
#Figure 3: Scenario ROC curve (multiple evaluations)
# System curve - Replacement, band-pass
# Band-pass scenario [does not require multiple evaluations]
# Replacement scenario
specs <- c(seq(0, 0.99, 0.01), seq(0.991, 0.995, 0.001))  # use `specs <- seq(0, 0.99, 0.04)` for example data
n_iter <- 50
res_repl_curve_multiple <- specs |> lapply(function(spec) {
    message("Processing: ", spec)
    suppressMessages({
        threshold <- find_manufacturer_threshold_matching(model_df, "specificities", spec)
    })
    pb <- txtProgressBar(0, n_iter, style = 3)
    1:n_iter |> lapply(function(iter) {
        setTxtProgressBar(pb, value = iter)
        suppressMessages({
            result <- run_simulation(
                run_AI_replacement,
                accession_df = accession_df,
                reader_df = reader_df,
                model_df = model_df,
                threshold = threshold,
                seed = seeds[iter],
                custom = custom
            )
        })
        unlink(paste0(normalizePath(tempdir()), "/", dir(tempdir())), recursive=TRUE)
        cbind(iter = iter, spec = spec, data.frame(result$system_econ))
    }) |> do_rbind()
}) |>
    do_rbind() |>
    data.frame()

In [ ]:
res_repl_curve_multiple
write.csv(res_repl_curve_multiple, file = output("res_repl_curve_multiple.csv"), row.names = FALSE)

In [ ]:
# Confirm there is no variation between runs for band-pass
write.csv(res_bp_curve_multiple, file = output("res_bp_curve_multiple.csv"), row.names = FALSE)
res_bp_curve_multiple <- read.csv(output("res_bp_curve_multiple.csv"))
res_bp_curve_multiple |> filter(iter == 1) |> head()
res_bp_curve_multiple |> filter(iter == 2) |> head()

In [ ]:
#Supplementary Table: scenario details
# Supplementary Table: scenario details ========================================
supp_table_repl(colMeans(res_repl$econ), res_baseline_econ) |> cat()
supp_table_bp(res_bp$system_econ, res_baseline_econ) |> cat()
supp_table_masai(res_masai_econ, res_baseline_econ) |> cat()

In [ ]:
#Supplementary Table: sensitivity analysis
# Supplementary Table: sensitivity analysis ====================================
# Second reader arbiter
second_reader_result <- run_simulation(
  BRAIxMOP:::run_AI_replacement_second_reader_arbiter,
  accession_df = accession_df,
  reader_df = reader_df,
  model_df = model_df,
  threshold = ai_performance$threshold,
  seed = 1234,
  custom = c(emp_fpr_3rd, emp_tpr_3rd, emp_tpr_3rd, emp_fpr_3rd, emp_fpr_3rd)
)

second_reader_result_n <- run_simulation_bootstrap(
  BRAIxMOP:::run_AI_replacement_second_reader_arbiter,
  accession_df = accession_df,
  reader_df = reader_df,
  model_df = model_df,
  threshold = ai_performance$threshold,
  custom = c(emp_fpr_3rd, emp_tpr_3rd, emp_tpr_3rd, emp_fpr_3rd, emp_fpr_3rd),
  run_parallel = FALSE,
  mc.cores = 4,
  seed = 1234,
  bootstrap_n = 1000
)

save_RDS(second_reader_result_n,
         output("simulation_replacement_second_reader_arbiter.RDS"),
         "Simulation result of the replacement scenario with second reader arbiter")


# Mixed third reader
mixed_third_result <- run_simulation(
  BRAIxMOP:::run_AI_replacement_mixed_third_reader,
  accession_df = accession_df,
  reader_df = reader_df,
  model_df = model_df,
  threshold = ai_performance$threshold,
  seed = 1234,
  custom = c(emp_fpr_3rd, emp_tpr_3rd, emp_tpr_3rd, emp_fpr_3rd, emp_fpr_3rd)
)

mixed_third_result_n <- run_simulation_bootstrap(
  BRAIxMOP:::run_AI_replacement_mixed_third_reader,
  accession_df = accession_df,
  reader_df = reader_df,
  model_df = model_df,
  threshold = ai_performance$threshold,
  custom = c(emp_fpr_3rd, emp_tpr_3rd, emp_tpr_3rd, emp_fpr_3rd, emp_fpr_3rd),
  run_parallel = FALSE,
  mc.cores = 3,
  seed = 1234,
  bootstrap_n = 1000
)

save_RDS(mixed_third_result_n,
         output("simulation_replacement_mixed_third_reader.RDS"),
         "Simulation result of the replacement scenario with mixed simulated third reader")

cat(supp_table_sensitivity_test(res_baseline_econ))

### Hypothesis testing

In [ ]:
#Hypothesis testing: AI-integrative scenario against AI baseline
# Testing superiority / non-inferiority of the AI integrative scenario w.r.t. the AI baseline ----
accession_df <- readRDS(output("accession_df.RDS"))
reader_df <- readRDS(output("reader_df.RDS"))
model_df <- readRDS(output("model_df.RDS"))
result_baseline <- baseline(accession_df, reader_df)
result_bp <- readRDS(output("simulation_band_pass.RDS"))
result_rr <- readRDS(output("simulation_replacement_original.RDS"))
result_tr <- readRDS(output("simulation_masai.RDS"))
set.seed(1234)
result_standalone <- model_df |>
    apply_manufacturer_threshold(ai_performance$threshold) |>
    select(episode_id, episode_outcome, episode_prediction)


# Match the median (Youden) case for Reader Replacement
result_rr_median <- local({
  # Find the median (Youden) case
  result_rr_df0 <- result_rr$performance |>
    mutate(idx = 1:1000,
           youden = TPR - FPR,
           youden_check = TPR + TNR - 1)
  yd <- result_rr_df0$youden
  stopifnot(any(yd == quantile(yd, 0.5, type = 1)))

  median_run <- result_rr_df0 |>
    filter(youden == quantile(yd, 0.5, type = 1))
  print(median_run)
  median_run$idx

  # Run the simulation
  set.seed(1234)
  n <- 1000
  multiple_runs_seed <- c(1234, sample(10000000, n - 1))
  median_seed <- multiple_runs_seed[median_run$idx]

  emp_perf <- get_reader_summary(reader_df)$mean_reader_by_position  # Test performance
  emp_tpr_3rd <- emp_perf$weighted_tpr[3]
  emp_fpr_3rd <- emp_perf$weighted_fpr[3]

  result <- run_simulation(
    run_AI_replacement,
    accession_df = accession_df,
    reader_df = reader_df,
    model_df = model_df,
    threshold = readRDS(output("AI_performance.RDS"))$threshold,
    seed = median_seed,
    custom = c(emp_fpr_3rd, emp_tpr_3rd, emp_tpr_3rd, emp_fpr_3rd, emp_fpr_3rd)
  )
  result
})
rr_pair <- pair_scenarios(result_baseline, result_rr_median$system_summary$result)
bp_pair <- pair_scenarios(result_baseline, result_bp$system_summary$result)
tr_pair <- pair_scenarios(result_baseline, result_tr)
standalone_pair <- pair_scenarios(result_baseline, result_standalone)


# McNemar test
run_test(bp_pair)
run_test(tr_pair)
run_test(rr_pair)
run_test(standalone_pair)